In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import math
import os

In [2]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
tf.reset_default_graph()

batch_size = 128
n_batch = math.ceil(mnist.train.num_examples / batch_size)

inputs = tf.placeholder(tf.float32, [None,784])
labels = tf.placeholder(tf.int32, [None,10])

# model
hidden1 = tf.layers.dense(inputs, 128, activation=tf.nn.relu, name='hidden-1')
hidden2 = tf.layers.dense(hidden1, 32, activation=tf.nn.relu, name='hidden-2')
logits = tf.layers.dense(hidden2, 10, activation=None, name='logits')

# loss + train_op
loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(labels, logits=logits))
global_step = tf.Variable(0, name='global_step', trainable=False)
learning_rate = tf.Variable(0.01,trainable=False,name="learing_rate")
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

prediction_index = tf.argmax(tf.nn.softmax(logits),axis=1)
label_index = tf.argmax(labels,axis=1)
bools_list = tf.equal(prediction_index,label_index)
accuracy = tf.reduce_mean(tf.cast(bools_list,tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
loss_summary = tf.summary.scalar("Loss",loss)
accuracy_summary = tf.summary.scalar("Accuracy",accuracy)
learning_rate_summary = tf.summary.scalar("Learning_rate",learning_rate)
test_text_summary = tf.summary.text("Test_text",tf.convert_to_tensor("hello tensorflow"))
summary_merge = tf.summary.merge([loss_summary,accuracy_summary,learning_rate_summary,test_text_summary])

In [5]:
from tensorflow.python.profiler import model_analyzer
from tensorflow.python.profiler import option_builder

In [6]:
run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()

graph = tf.get_default_graph()
profiler = model_analyzer.Profiler(graph=graph)
with tf.Session(graph=graph) as sess:
    sess.run(init)
    with tf.summary.FileWriter("./profiler/",graph=sess.graph,session=sess) as logWriter:
        for loop in range(100):
            for batch in range(n_batch):
                batch_xs,batch_ys = mnist.train.next_batch(batch_size)
                _ = sess.run(train_op,feed_dict={inputs:batch_xs,labels:batch_ys})
            else:
                test_acc,test_loss = sess.run([accuracy,loss],feed_dict = {inputs:mnist.test.images,labels:mnist.test.labels})
                train_acc,train_loss,summary = sess.run([accuracy,loss,summary_merge],feed_dict = {inputs:mnist.train.images,labels:mnist.train.labels},options=run_options, run_metadata=run_metadata)
                profiler.add_step(step=loop, run_meta=run_metadata)
                logWriter.add_summary(summary,loop)
                logWriter.add_run_metadata(run_metadata, 'step%d' % loop)
                if loop % 5 == 0:
                    print("loop = {},  test_acc = {},   train_acc = {},   test_loss={},  train_loss={}".format(loop,test_acc,train_acc,test_loss,train_loss))

Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes


31 ops no flops stats due to incomplete shapes.


loop = 0,  test_acc = 0.8414999842643738,   train_acc = 0.8334909081459045,   test_loss=0.6601693630218506,  train_loss=0.69017094373703
loop = 5,  test_acc = 0.9186999797821045,   train_acc = 0.9150182008743286,   test_loss=0.281646192073822,  train_loss=0.29932457208633423
loop = 10,  test_acc = 0.9365000128746033,   train_acc = 0.9339818358421326,   test_loss=0.22632412612438202,  train_loss=0.23457294702529907
loop = 15,  test_acc = 0.9458000063896179,   train_acc = 0.9449818134307861,   test_loss=0.18923373520374298,  train_loss=0.1941930055618286
loop = 20,  test_acc = 0.9520999789237976,   train_acc = 0.9525818228721619,   test_loss=0.1668083667755127,  train_loss=0.1673302948474884
loop = 25,  test_acc = 0.9562000036239624,   train_acc = 0.9589272737503052,   test_loss=0.1504470407962799,  train_loss=0.14683011174201965
loop = 30,  test_acc = 0.9592000246047974,   train_acc = 0.9631636142730713,   test_loss=0.13751156628131866,  train_loss=0.13136352598667145
loop = 35,  test_a

In [35]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances
#print(dir(notebook.manager))
notebook.start("--logdir=./profiler/ --host=127.0.0.1 --port=6006")  #在windows下会卡主一定时间，建议还是在外部使用命令行启动好tensorboard
#notebook.display(port=6006, height=1000) 

Known TensorBoard instances:
  - port 6006: logdir ./ (started 5:58:36 ago; pid 1096)
  - port 6006: logdir ./profiler/ (started 0:02:49 ago; pid 12920)
  - port 6006: logdir ./ (started 6:52:22 ago; pid 1356)
  - port 6006: logdir ./ (started 6:22:47 ago; pid 14076)
  - port 6006: logdir ./ (started 0:31:49 ago; pid 18648)
  - port 6006: logdir ./ (started 5:16:57 ago; pid 19100)
  - port 6006: logdir ./ (started 4:40:02 ago; pid 20896)
  - port 6006: logdir ./ (started 6:31:03 ago; pid 22100)
  - port 6006: logdir ./ (started 6:43:06 ago; pid 224)
  - port 6006: logdir ./ (started 6:13:44 ago; pid 2928)
  - port 6006: logdir ./ (started 1:53:32 ago; pid 5020)
  - port 6006: logdir ./ (started 5:14:18 ago; pid 5936)
  - port 6006: logdir ./ (started 6:26:49 ago; pid 6148)
  - port 6006: logdir ./ (started 4:38:38 ago; pid 6196)
  - port 6006: logdir ./profiler/ (started 0:28:45 ago; pid 6380)
  - port 6006: logdir ./ (started 1:29:14 ago; pid 9004)


Reusing TensorBoard on port 6006 (pid 12920), started 0:02:49 ago. (Use '!kill 12920' to kill it.)

In [41]:
#graph view显示每一个节点的用时，产生timeline
TIME_LINE_JSON_DIR = "./tmp/"
TIME_LINE_JSON_FILE_NAME = "timeline.json"

profile_graph_opts_builder = option_builder.ProfileOptionBuilder(
  option_builder.ProfileOptionBuilder.time_and_memory())

os.makedirs(TIME_LINE_JSON_DIR,exist_ok=True)
#输出方式为timeline
# 输出文件夹必须存在
profile_graph_opts_builder.with_timeline_output(timeline_file=os.path.join(TIME_LINE_JSON_DIR,TIME_LINE_JSON_FILE_NAME))
#定义显示sess.Run() 第70步的统计数据
profile_graph_opts_builder.with_step(49)

#显示视图为graph view
profiler.profile_graph(profile_graph_opts_builder.build())
pass

#使用Chrome浏览器使用chrome://tracing/打开json文件进行观看

In [102]:
#统计内容为所有trainable Variable Op
profile_scope_opt_builder = option_builder.ProfileOptionBuilder(
  option_builder.ProfileOptionBuilder.trainable_variables_parameter())

#显示的嵌套深度为4
profile_scope_opt_builder.with_max_depth(10)
#显示字段是params，即参数
profile_scope_opt_builder.select(['params'])
#根据params数量进行显示结果排序
profile_scope_opt_builder.order_by('params')

profile_scope_opt_builder.with_file_output("./params_count.pf")
##这里将导出结果设置为，导出到文件，默认是导出到stdout,可以在jupyter notebook中查看。

#显示视图为scope view
profiler.profile_name_scope(profile_scope_opt_builder.build())
pass

In [84]:
profile_op_opt_builder = option_builder.ProfileOptionBuilder()

#显示字段：op执行时间，使用该op的node的数量。 注意：op的执行时间即所有使用该op的node的执行时间总和。
profile_op_opt_builder.select(['micros','occurrence'])
#根据op执行时间进行显示结果排序
profile_op_opt_builder.order_by('micros')
#过滤条件：只显示排名top 5
profile_op_opt_builder.with_max_depth(4)

#显示视图为op view
profiler.profile_operations(profile_op_opt_builder.build())
pass
##执行之后能够在Jupyter Notebook的程序控制台中看到输出

In [88]:
profile_op_opt_builder = option_builder.ProfileOptionBuilder()

#显示字段：op占用内存，使用该op的node的数量。 注意：op的占用内存即所有使用该op的node的占用内存总和。
profile_op_opt_builder.select(['bytes','occurrence'])
#根据op占用内存进行显示结果排序
profile_op_opt_builder.order_by('bytes')
#过滤条件：只显示排名最靠前的5个op
profile_op_opt_builder.with_max_depth(4)

#显示视图为op view
profiler.profile_operations(profile_op_opt_builder.build())
pass
##执行之后能够在Jupyter Notebook的程序控制台中看到输出

In [92]:
#显示视图为code view(代码层面)
#profiler.profile_python